In [1]:
2+2

4

In [2]:
selector = 'listings-wrapper xs-flex xs-flex-wrap xs-grid sm-grid-cols-2 lg-grid-cols-3 xl-grid-cols-4 xs-gap-2 sm-gap-3 xs-mx2- sm-mx0'

print(f'Copy html element with the selector,')
print(f'"{selector}"')

Copy html element with the selector,
"listings-wrapper xs-flex xs-flex-wrap xs-grid sm-grid-cols-2 lg-grid-cols-3 xl-grid-cols-4 xs-gap-2 sm-gap-3 xs-mx2- sm-mx0"


In [127]:
final_cols = ['app_num','street_num','street', 'city', 'prov', 
              'district', 'latitude', 'longitude', 'sold_date', 
              'dom', 'sold_price', 'listing_price', 'beds','baths', 
              'size', 'mls', 'brokerage', 'listing_date', 'full_address']

main_df_path = 'main_data/main_results.csv'

empty_main_df = pd.DataFrame(columns = final_cols)
empty_main_df.to_csv(main_df_path,index=False)

main_df_path = 'review_data/duplicated_df.csv'

empty_main_df = pd.DataFrame(columns = final_cols)
empty_main_df.to_csv(main_df_path,index=False)

columns = ['unit_num', 'street', 'city', 'prov', 
               'district', 'latitude', 'longitude', 
               'sold_date', 'dom', 'sold_price',
               'listing_price', 'beds', 'baths', 
               'size', 'mls', 'brokerage']


main_df_path = 'review_data/check_coordinates.csv'

empty_main_df = pd.DataFrame(columns = columns)
empty_main_df.to_csv(main_df_path,index=False)

In [128]:
import os
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim


class extract_properties:
    
    def __init__(self):
        pass
        
    columns = ['unit_num', 'street', 'city', 'prov', 
               'district', 'latitude', 'longitude', 
               'sold_date', 'dom', 'sold_price',
               'listing_price', 'beds', 'baths', 
               'size', 'mls', 'brokerage']

        
    def to_dataframe(self, zolo_text, columns = columns):
        zolo_prop = zolo_text.split("data-propertyid")
        
        prop_data = pd.DataFrame(columns=columns)
        
        for prop in zolo_prop:
            if "MLS" in prop:
                row = {}
                details = prop.split("&")

                unit_num = details[13].split(';')[1]

                street = details[15].split(';')[1]

                city = details[19].split(';')[1]

                prov = details[23].split(';')[1]

                district = details[35].split(';')[1]

                #latitude
                latitude = details[42].split('=')[1].split()[0][1:-1]


                #longitude
                longitude = details[44].split('=')[1].split()[0][1:-1]

                # sold date
                sold_date = details[51].split('•')[1]

                # days on market
                dom = details[55].split('"p1">')[1]

                # sold price
                sold_price = details[63].split('$')[1]
                
                # listing price
                listing_price = details[67].split('$')[1]

                # num of beds
                beds = details[75].split(';')[1]

                # num of baths
                baths = details[79].split(';')[1]

                # size info may not be available, subsequent detail index will change
                # check if size detail exists in listing, 600-699 sqft or 1500-2000 sqft
                if ('99 sqft' in prop) or ('00 sqft' in prop):
                    size = details[83].split(';')[1]
                    mls = details[91].split(';')[1]
                    brokerage = details[99].split(';')[1]

                else:
                    size = np.NaN
                    mls = details[87].split(';')[1]
                    brokerage = details[95].split(';')[1]


                row = [unit_num, street, city, prov, district, latitude, longitude, 
                       sold_date, dom, sold_price, listing_price, beds, baths, size, mls, brokerage]

                row = pd.DataFrame([row], columns = columns)

                prop_data = pd.concat([prop_data, row])
                
        
        # remove all unwanted spaces in each columns
        def strip_string(value):
            # Check if the value is NaN (None) and return it as is
            if pd.isna(value):
                return value
            # Otherwise, apply the strip() method to remove leading and trailing whitespaces
            return value.strip()
        
        prop_data = prop_data.applymap(strip_string)   

        property_df = prop_data.reset_index(drop=True)
        
        print("Successfully converted to a dataframe")
        
        return property_df
    
    def clean_beds(self, property_df):
        # remove the bd suffixes
        property_df['beds'] = property_df['beds'].apply(lambda x: x.split()[0])
        print("Successfully extracted beds")
       
    
    def clean_baths(self, property_df):
        # remove the ba suffixes
        property_df['baths'] = property_df['baths'].apply(lambda x: x.split()[0])
        print("Successfully extracted baths")
        
    def clean_dom(self, property_df):
        
        empty_rows = None
        
        if '' in property_df['dom'].unique():
            empty_rows_index = property_df[property_df['dom']==''].index
            print(f'Check rows on website with empty DOMs is in fact 0: {empty_rows_index}')
            
            # replace empty rows with empty strings
            raw_df['dom'] = raw_df['dom'].replace('', '0 days on market')

        
        try:
            property_df['dom'] = property_df['dom'].apply(lambda x: x.split()[0])
            property_df['dom'] = pd.to_numeric(property_df['dom'])
            print("Successfully extracted DOMs with no concerns")
        
        except:
            print('DOMs has an unexpected value')
            
                
    def clean_sold_date(self, property_df):
        def get_sold_date(date, currentYear=2023):
            months_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 
                           'Jun', 'Jul', 'Aug', 'Sep', 'Oct']
            month = date.split()[0]
            month_num = months_list.index(month)

            date_num = date.split()[1].split('th')[0]
            return pd.to_datetime(f'{currentYear}-{month_num+1}-{date_num}')

        property_df['sold_date'] = property_df['sold_date'].apply(lambda x: get_sold_date(x))
        
        print("Successfully extracted sold date")

    def clean_prices(self, property_df):
        
        """
        Remvoe the comma and change price column datatypes to numeric
        """
        def remove_comma(x):
            """
            Remove the comma in he prices
            Input: '625,000'
            Ouput: '625000'
            """
            number = ''
            for i in x.split(','):
                number += i
            return number
        
        property_df[['sold_price','listing_price']] = property_df[['sold_price','listing_price']].applymap(remove_comma)
            
        property_df[['sold_price','listing_price']] = property_df[['sold_price','listing_price']].applymap(pd.to_numeric)
        
        print("Successfully converted price data from string to numeric")
            
    ## make function, add_price_Difference
    def add_listing_date(self, property_df):
        # get listing date
        listing_date = property_df['sold_date'] - pd.to_timedelta(property_df['dom'],
                                                                  unit='days')
        # add to main dataset
        property_df['listing_date'] = listing_date
        print("Successfully calculated listing date")
        
    def estimate_size(self, property_df):
        # clean unit sizes
        size_range = property_df['size'].apply( lambda x: x.split()[0] if type(x) == str else x)

        # get average size of range
        size = size_range.apply(lambda x: ( int(x.split('-')[0]) + int(x.split('-')[1]) ) // 2 if type(x) == str else x )

        # round off size to end in 00.0, ie. 500
        size = size.apply(lambda x: x + 1 if x%2 == 1 else x)
        size = size.astype('Int64')

        property_df['size'] = size
        print("Successfully calculated size")
        
    def add_street_num(self, property_df):
        # get the street number
        property_df['street_num'] = property_df['unit_num'].apply(lambda x: x.split('-')[-1])
        print("Successfully extracted street number")

    def add_app_num(self, property_df):
        # if has no apartment number give the appartment number -999
        property_df['app_num'] = property_df['unit_num'].apply(lambda x: x.split('-')[0] if len(x.split('-'))>1 else -999)
        print("Successfully extracted appartment number")
        
    def add_full_address(self, property_df):
        # get lat and long coordinates from full address
        property_df['full_address'] = property_df[['street_num','street','city','prov']].apply(lambda row: ' '.join(row), axis=1)
        print("Successfully extracted full address")
        
    def get_coordinates(self, address):
            """
            Helper function for function check_geo_coordinates to get latitude and longitude coordinates 
            from the library geopy.
            Input the address and return the latitude and longitude.
            """
            # Initialize a geocoder with the Nominatim service
            geolocator = Nominatim(user_agent="forward_geocoder")

            # Use the geocoder to get the location details (longitude and latitude)
            location = geolocator.geocode(address)

            if location:
                latitude = location.latitude
                longitude = location.longitude
                return latitude, longitude
            
        
    def check_geo_coordinates(self, property_df):
        """
        This function compares the zolo coordinates and geopy coorindates.
        If the sets of coordinates differ by 0.1 degree in either direction, 
            then return those rows for further review.
            
        0.001 degree is approximately 0.111km = 111 meters
        """
        
        # call on another class function to get coorindates
        geo_coords = pd.DataFrame(tuple(property_df['full_address'].apply(self.get_coordinates )), columns=['lat','long'])
            
        # update latitude and longitude data type
        property_df[['latitude','longitude']] = property_df[['latitude','longitude']].apply(pd.to_numeric)

        # compare the coordinates
        coords_df =  pd.DataFrame(columns =['lat_diff', 'long_diff'])

        coords_df['lat_diff'] = np.abs(property_df['latitude'] - geo_coords['lat'])
        coords_df['long_diff'] = np.abs(property_df['longitude'] - geo_coords['long'])

        # 0.1 diff in lat is 11.1km 
        rows_to_check = coords_df[(coords_df['lat_diff'] > 0.001) | (coords_df['long_diff'] > 0.1)].index
        
        # df of rows to check
        check_geo_df = property_df.iloc[rows_to_check]
        
        # master df of geo issues
        check_geo_df_path = 'review_data/check_coordinates.csv'
        main_check_geo_df = pd.read_csv(check_geo_df_path)
        
        main_check_geo_df = self.concat_df(main_check_geo_df, check_geo_df)
        
        main_check_geo_df.to_csv(check_geo_df_path, index=False)
        print("Completed adding rows that needs to be checked for coordinate issues")
    
    # final columns
    final_cols = ['app_num','street_num','street', 'city', 'prov', 
              'district', 'latitude', 'longitude', 'sold_date', 
              'dom', 'sold_price', 'listing_price', 'beds','baths', 
              'size', 'mls', 'brokerage', 'listing_date', 'full_address']
    
    main_df_path = 'main_data/main_results.csv'
      
    global main_df
    main_df = pd.read_csv(main_df_path)
    

    def important_cols(self, df):
        """
        df is the processed data and we want to reduce the columns to only the cleaned columns
        """
        return df[final_cols]
         
    def check_duplicates(self, new_df):
        """
        Input:
            main_df: master df
            new_df: newly created df
            
        Checks the full address and MLS number for duplicates.
        If there are duplicates, store them if a different database for manual review.
        """
        
        # construct df of info to check
        check_df = pd.DataFrame()
        check_df['main_mls'] = main_df['mls']
        check_df['incoming_mls'] = new_df['mls']
        
        check_df['main_full_address'] = main_df['full_address']
        check_df['incoming_full_address'] = new_df['full_address']
        
        dup_df = check_df.copy()

        # perform check if MLS nums or full addresses are the same
        dup_df = dup_df[ (check_df['main_mls']==check_df['incoming_mls']) | (check_df['main_full_address']==check_df['incoming_full_address'])]

        # get rows that are duplicates
        duplicated_df = main_df[(main_df['mls'] == dup_df['main_mls']) | 
                                (main_df['mls'] == dup_df['incoming_mls']) | 
                                (main_df['full_address'] == dup_df['main_full_address']) | 
                                (main_df['full_address'] == dup_df['incoming_full_address'])
                               ]
        
        duplicated_df = self.important_cols(duplicated_df)
        
        print(f"Duplicate df shape is {duplicated_df.shape}")
        
        dup_df_path = 'review_data/duplicated_df.csv'
            
        main_dup_df = pd.read_csv(dup_df_path)
        
        # add new rows to main dup df
        main_dup_df = self.concat_df(main_dup_df, duplicated_df)
        
        # save 
        main_dup_df.to_csv(dup_df_path, index=False)
        
        print("Completed duplicate check")
            
    
    def concat_df(self, main_df,new_df):
        """
        Input: main dataframe, newly processed dataframe
        Returns: combined dataframe for a new main df
        """
        
        return pd.concat([main_df,new_df])
    
    def add_to_main_df(self, new_df):
        
        new_df = self.important_cols(new_df)
            
        concat_df(main_df, new_df).to_csv(main_df_path, index=False)
        
        print("Added new df to main")

## Start workflow

In [129]:
html_file = 'raw_data/zolo_3.html'

with open(html_file, "r", encoding='utf-8') as f:
    text= f.read()

In [130]:
apple = extract_properties()
raw_df = apple.to_dataframe(text)

raw_df

Successfully converted to a dataframe


,unit_num,street,city,prov,district,latitude,longitude,sold_date,dom,sold_price,listing_price,beds,baths,size,mls,brokerage
0,28,Longbourne Drive,Toronto,ON,Willowridge-martingrove-richview,43.6797,-79.5699,Aug 17th,37 days on market,"1,235,000","1,275,000",3 bd,3 ba,NaN,W6653800,ROYAL LEPAGE NEW CONCEPT
1,302-15,Rean Drive,Toronto,ON,Bayview Village,43.7668,-79.3821,Aug 17th,56 days on market,"665,800","677,000",1 bd,1 ba,700-799 sqft,C6196916,LIVING REALTY INC.
2,103,Warwood Road,Toronto,ON,Islington-city Centre West,43.6534,-79.5571,Aug 17th,36 days on market,"3,500,000","3,698,000",5 bd,5 ba,3500-5000 sqft,W6657332,SOTHEBY`S INTERNATIONAL REALTY CANADA
3,332-251,Manitoba Street,Toronto,ON,Mimico,43.6236,-79.4894,Aug 17th,42 days on market,"780,000","799,900",2 bd,2 ba,800-899 sqft,W6643500,RE/MAX PROFESSIONALS INC.
4,603-10,Queens Quay,Toronto,ON,Waterfront Communities C1,43.6419,-79.3757,Aug 17th,98 days on market,"636,000","648,000",1 bd,1 ba,700-799 sqft,C5978092,ROYAL LEPAGE TERREQUITY REALTY
5,7,Grasspoint Crescent,Toronto,ON,Markland Wood,43.6363,-79.5752,Aug 16th,7 days on market,"1,562,777","1,399,999",3 bd,2 ba,NaN,W6727804,VANGUARD REALTY BROKERAGE CORP.
6,305-75,Scarborough Road,Toronto,ON,The Beaches,43.6756,-79.2847,Aug 16th,6 days on market,"535,000","529,000",2 bd,1 ba,800-899 sqft,E6731492,CHESTNUT PARK REAL ESTATE LIMITED
7,17,Nomad Crescent,Toronto,ON,Banbury-don Mills,0,0,Aug 16th,1 day on market,"4,360,000","4,380,000",5 bd,5 ba,3500-5000 sqft,C6743806,LC REALTY INC.
8,110,Bertrand Avenue,Toronto,ON,Ionview,43.7364,-79.2757,Aug 16th,5 days on market,"1,005,000","999,000",3 bd,3 ba,NaN,E6734624,RE/MAX CROSSROADS REALTY INC.
9,703-5001,Finch Avenue,Toronto,ON,Agincourt North,43.8078,-79.2678,Aug 16th,5 days on market,"470,000","438,000",1 bd,1 ba,600-699 sqft,E6735154,SKYLAND REALTY INC.


In [131]:
raw_df.shape

(24, 16)

In [132]:
apple.clean_beds(raw_df)
apple.clean_baths(raw_df)
apple.clean_dom(raw_df)
apple.clean_sold_date(raw_df)
apple.clean_prices(raw_df)
apple.add_listing_date(raw_df)
apple.estimate_size(raw_df)
apple.add_street_num(raw_df)
apple.add_app_num(raw_df)
apple.add_full_address(raw_df)

# may require manual review because the street name is 
    # 1. does not match with an address in the geopy.geocoders database
    # 2. Wrong
        # 2.1 spelling error
        # 2.2 missing information
apple.check_geo_coordinates(raw_df)
apple.check_duplicates(raw_df)

Successfully extracted beds
Successfully extracted baths
Check rows on website with empty DOMs is in fact 0: Index([16, 22], dtype='int64')
Successfully extracted DOMs with no concerns
Successfully extracted sold date
Successfully converted price data from string to numeric
Successfully calculated listing date
Successfully calculated size
Successfully extracted street number
Successfully extracted appartment number
Successfully extracted full address
Completed adding rows that needs to be checked for coordinate issues
Duplicate df shape is (0, 19)
Completed duplicate check


In [30]:
raw_df.shape

(24, 20)

In [9]:
raw_df.iloc[[16,20],:]

,unit_num,street,city,prov,district,latitude,longitude,sold_date,dom,sold_price,listing_price,beds,baths,size,mls,brokerage,listing_date,street_num,app_num,full_address
16,66,Millhouse Crescent,Toronto,ON,Rouge E11,43.8052,-79.1742,2023-08-16,0,990000,995000,3,2,<NA>,E6748000,HOMELIFE TODAY REALTY LTD.,2023-08-16,66,-999,66 Millhouse Crescent Toronto ON
20,772,Markham Street,Toronto,ON,Annex,43.6688,-79.4141,2023-08-16,6,2265000,1849900,4,3,<NA>,C6734188,RE/MAX HALLMARK ESTATE GROUP REALTY LTD.,2023-08-10,772,-999,772 Markham Street Toronto ON


In [10]:
# # may require manual review because the street name is 
#     # 1. does not match with an address in the geopy.geocoders database
#     # 2. Wrong
#         # 2.1 spelling error
#         # 2.2 missing information
# apple.check_geo_coordinates(raw_df)

,unit_num,street,city,prov,district,latitude,longitude,sold_date,dom,sold_price,listing_price,beds,baths,size,mls,brokerage,listing_date,street_num,app_num,full_address
7,17,Nomad Crescent,Toronto,ON,Banbury-don Mills,0.0000,0.0000,2023-08-16,1,4360000,4380000,5,5,4250,C6743806,LC REALTY INC.,2023-08-15,17,-999,17 Nomad Crescent Toronto ON
9,703-5001,Finch Avenue,Toronto,ON,Agincourt North,43.8078,-79.2678,2023-08-16,5,470000,438000,1,1,650,E6735154,SKYLAND REALTY INC.,2023-08-11,5001,703,5001 Finch Avenue Toronto ON


In [11]:
# correct the coordinates with the manually reviewed address
# addreseses below are only for zolo.html file

if html_file == 'raw_data/zolo.html':
    correct_coords_1 = apple.get_coordinates('847 Sheppard Avenue W Toronto ON')
    correct_coords_2 = apple.get_coordinates('66 Forest Manor Road Toronto ON')

    raw_df.loc[1, ['latitude','longitude']]=correct_coords_1
    raw_df.loc[1, ['latitude','longitude']]=correct_coords_2
    raw_df

In [ ]:
raw_df['size'].isna().sum()

In [ ]:
raw_df['size'].isna().sum()

In [ ]:
correct_coords_1

In [12]:
raw_df

,unit_num,street,city,prov,district,latitude,longitude,sold_date,dom,sold_price,listing_price,beds,baths,size,mls,brokerage,listing_date,street_num,app_num,full_address
0,28,Longbourne Drive,Toronto,ON,Willowridge-martingrove-richview,43.6797,-79.5699,2023-08-17,37,1235000,1275000,3,3,<NA>,W6653800,ROYAL LEPAGE NEW CONCEPT,2023-07-11,28,-999,28 Longbourne Drive Toronto ON
1,302-15,Rean Drive,Toronto,ON,Bayview Village,43.7668,-79.3821,2023-08-17,56,665800,677000,1,1,750,C6196916,LIVING REALTY INC.,2023-06-22,15,302,15 Rean Drive Toronto ON
2,103,Warwood Road,Toronto,ON,Islington-city Centre West,43.6534,-79.5571,2023-08-17,36,3500000,3698000,5,5,4250,W6657332,SOTHEBY`S INTERNATIONAL REALTY CANADA,2023-07-12,103,-999,103 Warwood Road Toronto ON
3,332-251,Manitoba Street,Toronto,ON,Mimico,43.6236,-79.4894,2023-08-17,42,780000,799900,2,2,850,W6643500,RE/MAX PROFESSIONALS INC.,2023-07-06,251,332,251 Manitoba Street Toronto ON
4,603-10,Queens Quay,Toronto,ON,Waterfront Communities C1,43.6419,-79.3757,2023-08-17,98,636000,648000,1,1,750,C5978092,ROYAL LEPAGE TERREQUITY REALTY,2023-05-11,10,603,10 Queens Quay Toronto ON
5,7,Grasspoint Crescent,Toronto,ON,Markland Wood,43.6363,-79.5752,2023-08-16,7,1562777,1399999,3,2,<NA>,W6727804,VANGUARD REALTY BROKERAGE CORP.,2023-08-09,7,-999,7 Grasspoint Crescent Toronto ON
6,305-75,Scarborough Road,Toronto,ON,The Beaches,43.6756,-79.2847,2023-08-16,6,535000,529000,2,1,850,E6731492,CHESTNUT PARK REAL ESTATE LIMITED,2023-08-10,75,305,75 Scarborough Road Toronto ON
7,17,Nomad Crescent,Toronto,ON,Banbury-don Mills,0.0000,0.0000,2023-08-16,1,4360000,4380000,5,5,4250,C6743806,LC REALTY INC.,2023-08-15,17,-999,17 Nomad Crescent Toronto ON
8,110,Bertrand Avenue,Toronto,ON,Ionview,43.7364,-79.2757,2023-08-16,5,1005000,999000,3,3,<NA>,E6734624,RE/MAX CROSSROADS REALTY INC.,2023-08-11,110,-999,110 Bertrand Avenue Toronto ON
9,703-5001,Finch Avenue,Toronto,ON,Agincourt North,43.8078,-79.2678,2023-08-16,5,470000,438000,1,1,650,E6735154,SKYLAND REALTY INC.,2023-08-11,5001,703,5001 Finch Avenue Toronto ON
